In [22]:
import plotly.graph_objects as go
import plotly.express as px
from dash.dependencies import Input, Output
import dash_table
import networkx as nx
import pandas as pd
import numpy as np
import json
import pickle 

from app import app

# needed only if running this as a single page app
#external_stylesheets = [dbc.themes.LUX]

#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# change to app.layout if running as single page app instead

graph_style = {"maxHeight": "auto", "align":"center", "margin": "auto"}

# Read data
df = pd.read_csv('HP_enriched_character_df.csv')

with open("G.pickle", 'rb') as f:
    G = pickle.load(f)

with open("GCC.pickle", 'rb') as f:
    GCC = pickle.load(f)


/var/folders/bg/jsf24hsj1ln_rl_40sb3wnj00000gn/T/ipykernel_59369/3816901239.py:4: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table


In [187]:
in_degree_vals = list(nx.get_node_attributes(G,'in_degree').values())
out_degree_vals = list(nx.get_node_attributes(G,'out_degree').values())

# Get hist values
hist_in, bins_in = np.histogram(in_degree_vals, bins = np.arange(np.min(in_degree_vals), np.max(in_degree_vals) + 2))
centered_in = (bins_in[:-1] + bins_in[1:]) / 2
hist_out, bins_out = np.histogram(out_degree_vals, bins = np.arange(np.min(out_degree_vals), np.max(out_degree_vals) + 2))
centered_out = (bins_out[:-1] + bins_out[1:]) / 2

df_in_degree = pd.DataFrame({"In-degree": hist_in, "Count": centered_in})
df_out_degree = pd.DataFrame({"Out-degree": hist_out, "Count": centered_out})

# Make figure
fig = go.Figure(make_subplots(rows=2, cols=1))

marker_layout_in = dict(size=8,color = "blue", line_width=1,opacity = 0.5)
marker_layout_out = dict(size=8,color = "red", line_width=1,opacity = 0.5)

# Add traces
fig.add_trace(go.Scatter(x=df_in_degree["Count"], y=df_in_degree["In-degree"],
                    mode='markers',
                    marker=marker_layout_in,
                    hoverinfo='none',
                    name='In-degree'), 
            row=1, col=1
            )

fig.add_trace(go.Scatter(x=df_out_degree["Count"], y=df_out_degree["Out-degree"],
                    mode='markers',
                    marker=marker_layout_out,
                    hoverinfo='none',
                    name='Out-degrees'),
            row=1, col=1
            )


fig.add_trace(go.Scatter(x=df_in_degree["Count"], y=df_in_degree["In-degree"],
                mode='markers',
                    marker=marker_layout_in,
                    hoverinfo='none',
                    name='In-degree',
                    showlegend  =False), 
            row=2, col=1
            )

fig.add_trace(go.Scatter(x=df_out_degree["Count"], y=df_out_degree["Out-degree"],
                    mode='markers',
                    marker=marker_layout_out,
                    hoverinfo='none',
                    name='Out-degrees',
                    showlegend  =False),
            row=2, col=1
            )

# Update xaxis properties
fig.update_xaxes(title_text="Degree", row=1, col=1)
fig.update_xaxes(title_text="log(Degree)", row=2, col=1, type="log")

# Update yaxis properties
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="log(Count)", row=2, col=1, type="log")

# Update title and height
fig.update_layout(title_text="Degree distribution", height=800)

fig.update_layout(
    legend=dict(
        font=dict(
            #family="Courier",
            size=18,
            color="black"
        )),
    title=dict(
        font=dict(
            #family="Courier",
            size=20,
            color="black"
        )
    )
    )

## To cyto file

In [188]:
# Update positions
pos = nx.get_node_attributes(G,'pos')
pos = {i: np.array(p) for i, p in pos.items()}
nx.set_node_attributes(G,pos, 'pos')

pos = nx.get_node_attributes(GCC,'pos')
pos = {i: np.array(p) for i, p in pos.items()}
nx.set_node_attributes(GCC,pos, 'pos')

In [189]:
# Update positions (scale)
positions = nx.get_node_attributes(G,'pos')
positions.update((key, tuple([10000*x for x in value])) for key, value in positions.items())
nx.set_node_attributes(G, positions, 'pos')

positions = nx.get_node_attributes(GCC,'pos')
positions.update((key, tuple([10000*x for x in value])) for key, value in positions.items())
nx.set_node_attributes(GCC, positions, 'pos')

# Update size (scale)
positions = nx.get_node_attributes(G,'size')
positions.update((key, value) for key, value in positions.items())
nx.set_node_attributes(G, positions, 'size')

positions = nx.get_node_attributes(GCC,'size')
positions.update((key, value) for key, value in positions.items())
nx.set_node_attributes(GCC, positions, 'size')

In [190]:
cyto_GCC_json = nx.cytoscape_data(GCC)['elements']

for node in cyto_GCC_json['nodes']:
    node.update({'classes': node['data']['house']})
    node.update({'position': {"x":node['data']['pos'][0],"y":node['data']['pos'][1]}})
    node.update({'size': node['data']['size']})
    
import json
with open('cyto_GCC.json', 'w') as outfile:
       json.dump(cyto_GCC_json, outfile)
        
cyto_full_network_json = nx.cytoscape_data(G)['elements']

for node in cyto_full_network_json['nodes']:
    node.update({'classes': node['data']['house']})
    node.update({'position': {"x":node['data']['pos'][0],"y":node['data']['pos'][1]}})
    node.update({'size': node['data']['size']})
                
import json
with open('cyto_full_network.json', 'w') as outfile:
       json.dump(cyto_full_network_json, outfile)

In [191]:
with open("cyto_full_network.json", 'rb') as f:
    cyto_G = json.load(f)

with open("cyto_GCC.json", 'rb') as f:
    cyto_GCC = json.load(f)



In [193]:
cyto_G['nodes'][1]['position']

{'x': -1424.1233995045666, 'y': -1491.447309906207}